<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

if os.path.exists(dep_filename):
    print(f"{dep_filename} is in your current directory. Assuming you are in the root of the repository.")
elif not os.path.exists(repo_name):
    print(f"Cloning repo and fetching {dep_filename}...")
    !git clone https://github.com/Sebelino/DD2424-project
    !cp DD2424-project/download_dataset.py .
else:
    !git -C DD2424-project pull --rebase
    !cp DD2424-project/download_dataset.py .

Host: deeplearning-vm
You are here: /home/silpasoninallacheruvu/DD2424-project
download_dataset.py is in your current directory. Assuming you are in the root of the repository.


In [5]:
import torch
torch.set_float32_matmul_precision('high')

# Train model

In [ ]:
# from dataclasses import asdict
# from util import json_dumps
# from evaluation import evaluate_test_accuracy_and_misclassified, \
#     evaluate_with_train_val_plot, run_with_different_seeds, run_comparison
# from run import run
# %reload_ext autoreload
# %autoreload 2

# from determinism import Determinism

# determinism = Determinism(seed=42).sow()  # Keep this before any torch import

# from training import TrainParams, NagParams, AdamParams

# baseline_params = TrainParams(
#     seed=determinism.seed,
#     batch_size=32,
#     architecture="resnet50",
#     n_epochs=10,
#     optimizer=NagParams(
#         learning_rate=0.01,
#         weight_decay=1e-4,
#         momentum=0.9,
#     ),
#     freeze_layers=True,
#     unfreezing_epochs=None,
#     validation_freq=1,
#     unfreeze_last_l_blocks = 1,
#     time_limit_seconds=None,
#     val_acc_target=None,
# )

# invalidate_cache_entry = True
# if invalidate_cache_entry:
#     run.call_and_shelve(baseline_params).clear()

# # First plot: Training and validation accuracy
# for l in range(1,6):
#     baseline_params.unfreeze_last_l_blocks = l
#     print("Training with params:")
#     print(baseline_params.pprint())
#     result = run(baseline_params)

#     print("Training result:")
#     print(result.pprint())
#     evaluate_with_train_val_plot(result)

#     # Second plot: Rerun training multiple times with different seed
#     run_with_different_seeds(baseline_params, trials=3)

#     # Third plot: Compare NAG with Adam
#     experimental_params = baseline_params.copy()
#     experimental_params.optimizer = AdamParams(learning_rate=0.001)
    
#     param_sets = {
#         "Baseline (NAG)": baseline_params,
#         "Experiment (Adam)": experimental_params,
#     }
    
#     run_comparison(param_sets, trials=3)


# Train semi-supervised model

In [ ]:
%reload_ext autoreload
%autoreload 2

from training import TrainParams, NagParams
from evaluation import evaluate_test_accuracy_and_misclassified, \
    evaluate_with_train_val_plot, run_with_different_seeds, run_comparison
from run import run
from caching import invalidate_cache_entry
from datasets import DatasetParams
from augmentation import AugmentationParams

from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 1,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

run_args = (dataset_params, baseline_params)

# Set invalidate=True to invalidate a single cache entry
invalidate_cache_entry(run, run_args, invalidate=False)

# First plot: Training and validation accuracy
print("Training with params:")
print(baseline_params.pprint())
result = run(*run_args)

print("Training result:")
print(result.pprint())
evaluate_with_train_val_plot(result)

In [ ]:
from evaluation import run_dataset_comparison

def tweak(params: DatasetParams, labelled_data_fraction):
    params = params.copy()
    params.labelled_data_fraction = labelled_data_fraction
    return params
 
#plot: Compare unsupervised learning for different fractions of unlabelled data
labelled_data_fractions = [ 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
param_sets = {f"labelled_data_fraction={data_fraction}": tweak(dataset_params, labelled_data_fraction=data_fraction) for data_fraction in labelled_data_fractions}

run_dataset_comparison(param_sets, baseline_params)


In [ ]:
from evaluation import run_dataset_comparison

def tweak(params: DatasetParams, labelled_data_fraction):
    params = params.copy()
    params.labelled_data_fraction = labelled_data_fraction
    return params
 
#plot: Compare unsupervised learning for different fractions of unlabelled data along with threshold
labelled_data_fractions = [ 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
param_sets = {f"labelled_data_fraction={data_fraction}": tweak(dataset_params, labelled_data_fraction=data_fraction) for data_fraction in labelled_data_fractions}

baseline_params.pseudo_threshold = None
run_dataset_comparison(param_sets, baseline_params)


In [5]:
from evaluation import evaluate_test_accuracy
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 1,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

#plot: Compare unsupervised learning for different values of unsup_weight
unsup_weights = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for unsup_weight in unsup_weights:
    baseline_params.unsup_weight = unsup_weight
    training_params = deepcopy(baseline_params)

    print("\n=====================================")
    print(f"Training with unsup_weight={unsup_weight}")
    print("=====================================")
    print(training_params.pprint())

    # trainer = try_loading_trainer(dataset_params, training_params, determinism)

    # test_dataset = load_dataset("test", Trainer.make_base_transform(training_params))
    # test_loader = DataLoader(
    #     test_dataset,
    #     batch_size=dataset_params.batch_size,
    #     shuffle=False,
    #     num_workers=3,
    #     persistent_workers=False,
    #     pin_memory=True,
    #     worker_init_fn=Determinism.data_loader_worker_init_fn(dataset_params.shuffler_seed),
    # )
    trials = 1
    # print(f"Test size: {len(test_loader.dataset)}")
    evaluate_final_test_accuracy(
            dataset_params,
            training_params,
            determinism,
            trials,
            display_misclassified=False,
        )
    # print(f"Final Test Accuracy: {final_test_acc:.2f}%")



Training with unsup_weight=0.1
{
  "seed": 42,
  "architecture": "resnet50",
  "optimizer": {
    "learning_rate": 0.004713842776302303,
    "weight_decay": 4.287389519996026e-06,
    "momentum": 0.7222321829989696,
    "name": "nag"
  },
  "n_epochs": 5,
  "freeze_layers": true,
  "unfreezing_epochs": [2, 4],
  "augmentation": {
    "enabled": false,
    "dropout_rate": 0.3
  },
  "validation_freq": 1,
  "mft": {
    "enabled": false,
    "k": 0,
    "impl": "theirs"
  },
  "unfreeze_last_l_blocks": 2,
  "fixmatch": true,
  "unsup_weight": 0.1,
  "pseudo_threshold": 0.95,
  "contrastive_temp": 0.1,
  "num_classes": 37,
  "use_scheduler": true,
  "scheduler_type": "plateau"
}
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/580 [00:00<?, ?it/s]

Epoch [1/5], Loss: 3.1379, Train Acc: 39.11%
Epoch [2/5], Loss: 1.5996, Train Acc: 83.70%
Epoch [3/5], Loss: 0.6819, Train Acc: 88.81%
Epoch [4/5], Loss: 0.4123, Train Acc: 92.67%
Epoch [5/5], Loss: 0.2976, Train Acc: 94.51%
Total elapsed: 114.84s, average per update step: 0.10s


...

Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/580 [00:00<?, ?it/s]

Epoch [1/5], Loss: 3.1379, Train Acc: 39.11%
Epoch [2/5], Loss: 1.5996, Train Acc: 83.70%
Epoch [3/5], Loss: 0.6819, Train Acc: 88.81%
Epoch [4/5], Loss: 0.5341, Train Acc: 92.29%
Epoch [5/5], Loss: 0.4033, Train Acc: 94.46%
Total elapsed: 114.35s, average per update step: 0.10s


Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Test Accuracy: 91.496 %
Test Accuracy Mean: 91.50 %


# Main Testing loop for semi-supervised model

In [7]:
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

%reload_ext autoreload
%autoreload 2


determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=1.0, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 0.5,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

fractions = [0.7, 0.5, 0.3, 0.1] #[0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
epochs_list = [10, 20, 30]

for fraction in fractions:
    dataset_params.labelled_data_fraction = fraction
    for num_epochs in epochs_list:
        training_params = deepcopy(baseline_params)
        training_params.n_epochs = num_epochs

        print("\n=====================================")
        print(f"Training with fraction={fraction}, epochs={num_epochs}")
        print("=====================================")
        # print(training_params.pprint())
        # print(dataset_params.pprint())

        trials = 3
        # run_with_different_seeds(dataset_params, training_params, determinism, trials)
        evaluate_final_test_accuracy(
            dataset_params,
            training_params,
            determinism,
            trials,
            display_misclassified=False,
        )


Training with fraction=0.7, epochs=10
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/1160 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.8437, Train Acc: 50.93%
Epoch [2/10], Loss: 1.0082, Train Acc: 84.57%
Epoch [3/10], Loss: 0.4513, Train Acc: 91.20%
Epoch [4/10], Loss: 0.3625, Train Acc: 94.26%
Epoch [5/10], Loss: 0.3110, Train Acc: 95.04%
Epoch [6/10], Loss: 0.4132, Train Acc: 96.43%
Epoch [7/10], Loss: 0.3774, Train Acc: 97.64%
Epoch [8/10], Loss: 0.3698, Train Acc: 96.51%
Epoch [9/10], Loss: 0.4130, Train Acc: 98.22%
Epoch [10/10], Loss: 0.3481, Train Acc: 98.10%
Total elapsed: 187.16s, average per update step: 0.10s


...

Test Accuracy: 82.502 %
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/3480 [00:00<?, ?it/s]

Epoch [1/30], Loss: 3.5497, Train Acc: 6.76%
Epoch [2/30], Loss: 3.3201, Train Acc: 31.62%
Epoch [3/30], Loss: 3.0601, Train Acc: 58.65%
Epoch [4/30], Loss: 2.7703, Train Acc: 77.57%
Epoch [5/30], Loss: 2.3964, Train Acc: 88.92%
Epoch [6/30], Loss: 1.9803, Train Acc: 92.70%
Epoch [7/30], Loss: 1.5701, Train Acc: 91.89%
Epoch [8/30], Loss: 1.1408, Train Acc: 94.86%
Epoch [9/30], Loss: 0.8907, Train Acc: 96.76%
Epoch [10/30], Loss: 0.6753, Train Acc: 96.22%
Epoch [11/30], Loss: 0.5430, Train Acc: 97.84%
Epoch [12/30], Loss: 0.4203, Train Acc: 98.65%
Epoch [13/30], Loss: 0.3623, Train Acc: 98.65%
Epoch [14/30], Loss: 0.3122, Train Acc: 99.19%
Epoch [15/30], Loss: 0.2576, Train Acc: 100.00%
Epoch [16/30], Loss: 0.2170, Train Acc: 99.73%
Epoch [17/30], Loss: 0.1795, Train Acc: 100.00%
Epoch [18/30], Loss: 0.1686, Train Acc: 99.73%
Epoch [19/30], Loss: 0.2998, Train Acc: 99.46%
Epoch [20/30], Loss: 0.1408, Train Acc: 99.73%
Epoch [21/30], Loss: 0.2625, Train Acc: 100.00%
Epoch [22/30], Loss:

Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Test Accuracy: 83.538 %
Test Accuracy Mean: 83.12 %
Test Accuracy Standard Error: 0.32 percentage points


In [3]:
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

%reload_ext autoreload
%autoreload 2


determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=1.0, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=20,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 0.5,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

fractions = [0.1, 0.3, 0.5, 0.7, 0.9]
pseudo_threshold_list = [None, 0.5, 0.7]

for fraction in fractions:
    dataset_params.labelled_data_fraction = fraction
    for pseudo_threshold in pseudo_threshold_list:
        training_params = deepcopy(baseline_params)
        training_params.pseudo_threshold = pseudo_threshold

        print("\n=====================================")
        print(f"Training with fraction={fraction}, pseudo_threshold={pseudo_threshold}")
        print("=====================================")
        # print(training_params.pprint())
        # print(dataset_params.pprint())


        trials = 3
        evaluate_final_test_accuracy(
            dataset_params,
            training_params,
            determinism,
            trials,
            display_misclassified=False,
        )


Training with fraction=0.7, pseudo_threshold=None
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/2320 [00:00<?, ?it/s]

Epoch [1/20], Loss: 3.3793, Train Acc: 35.23%
Epoch [2/20], Loss: 1.4076, Train Acc: 76.16%
Epoch [3/20], Loss: 0.7320, Train Acc: 86.78%
Epoch [4/20], Loss: 0.5728, Train Acc: 91.55%
Epoch [5/20], Loss: 0.5585, Train Acc: 94.84%
Epoch [6/20], Loss: 0.4148, Train Acc: 96.01%
Epoch [7/20], Loss: 0.3625, Train Acc: 97.21%
Epoch [8/20], Loss: 0.3759, Train Acc: 97.09%
Epoch [9/20], Loss: 0.3338, Train Acc: 98.14%
Epoch [10/20], Loss: 0.3856, Train Acc: 98.57%
Epoch [11/20], Loss: 0.4259, Train Acc: 99.03%
Epoch [12/20], Loss: 0.3985, Train Acc: 99.26%
Epoch [13/20], Loss: 0.3718, Train Acc: 99.03%
Epoch [14/20], Loss: 0.3757, Train Acc: 99.69%
Epoch [15/20], Loss: 0.3364, Train Acc: 99.26%
Epoch [16/20], Loss: 0.3470, Train Acc: 99.34%
Epoch [17/20], Loss: 0.3760, Train Acc: 99.26%
Epoch [18/20], Loss: 0.3594, Train Acc: 99.46%
Epoch [19/20], Loss: 0.3831, Train Acc: 99.57%
Epoch [20/20], Loss: 0.3452, Train Acc: 99.53%
Total elapsed: 389.51s, average per update step: 0.10s


...

Test Accuracy: 91.769 %
Test Accuracy Mean: 91.79 %
Test Accuracy Standard Error: 0.03 percentage points

Training with fraction=1.0, pseudo_threshold=None
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/2300 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from evaluation import show_misclassified

final_test_acc, misclassified_samples = evaluate_test_accuracy_and_misclassified(trainer, test_loader, test_dataset)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")
print(f"Number of misclassified samples: {len(misclassified_samples)}")
show_misclassified(misclassified_samples)

In [ ]:
from evaluation import show_misclassified

final_test_acc_tau, misclassified_samples_tau = evaluate_test_accuracy_and_misclassified(trainer, test_loader, test_dataset)
print(f"Final Test Accuracy: {final_test_acc_tau:.2f}%")
print(f"Number of misclassified samples: {len(misclassified_samples_tau)}")
show_misclassified(misclassified_samples_tau)

In [ ]:
misclassified_diff = list(set(misclassified_samples).symmetric_difference(misclassified_samples_tau))
print(f"Difference of misclassified samples: {len(misclassified_diff)}")
show_misclassified(misclassified_diff)

In [ ]:
misclassified_diff2 = list(set(misclassified_samples) - set(misclassified_samples_tau))
print(f"Difference of misclassified samples: {len(misclassified_diff2)}")
show_misclassified(misclassified_diff2)